# Model classification I – building blocks

<div style="text-align: center; font-style: italic;">

“A problem well stated is a problem half-solved.”

</div>

<div style="text-align: right;">

— Charles F. Kettering

</div>


## 1 What are models?  

### 1.1 From toy cathedrals to floating point arithmetic

<div style="text-align: center; font-style: italic;">
To measure is to know.
</div>

<div style="text-align: right;">

— William Thomson (Lord Kelvin), 1883

</div>


While we often think of models as mathematical equations, the mathematication of nature is a relatively recent development in the history of science and engineering. Models have existed in various forms for centuries, evolving from physical constructions to modern abstract mathematical representations. Some historical milestones in the practice of modelling include:

#### **Medieval (Physical (literal) models)**
  Medieval master-builders often erected stick-and-string scale models of vaults and domes to study thrust lines.  
  Brunelleschi’s herringbone brick pattern for the Florence Duomo (1430s) was tested on a wooden mock-up before construction.  
  These tangible artefacts embodied the equilibrium of forces as a precursor to statics theory wh 

#### **Early mathematical models (smooth models)**  
  - **Jordanus de Nemore** (13 th c.) treated levers and centres of gravity with geometric abstraction.  
  - **Leonhard Euler** (1740s) and **D’Alembert** introduced differential equations to describe beams, fluids, and celestial mechanics.  
    Euler’s equation for a buckling column is still taught today.

#### **Modern, discrete models (numerical models)**
  - Early hand computations by Isaac Newton and others laid the groundwork for numerical methods.  
  - First general purpose high level programming language Plankalkül (1942), Claude Shannon’s information theory (1948) and von Neumann’s stored-program computer made it routine to encode physics directly in software.

Our focus in this module is on discrete or digital models, which are fundamentally different to both physical and smooth models in a subtle and important way: while smooth mathematical models can often admit exact solutions, discretised models -by contrast- rarely admit exact solutions. This is not only due to numerical errors, but due to the fact that these models live in an entirely different space and number system. E.g. smooth models often have scalar variables represented by real numbers ($\mathbb{R}^n$) which is _not_ a computational structure! By contrast models for control systems often rely on a discrete $\mathbb{Z}^n$ representation of scalar variables arising from inherently discrete measurements in the control loop.

For the purposee of this course, we will define a model as follows:
> 
> **Definition:** A *model* is an abstraction which is deliberate simplification of reality that allows quantitative prediction.

In order to make predictions with *computational models*, it is essential to correclty classify them in order to find the correct abstracted method for their solutions which in turn is used to predict the outcome of systems such as the dynamics spacecraft, robots, or chemical processes, Later we will study the optimization (design, parameterisation) and control of systems.

---

## 2 Classification of models

Models admit different *structures* which can be classified in order to gain better understanding of their properities. Once a model has been classified it becomes easier to compute. Understanding the limitations of state-of-the-art methods used to solve different model types is essential to both model development and computational efficiency of solutions


### 2.1 Linear vs. non-linear  

|                    | Linear                                       | Non-linear                                    |
|--------------------|----------------------------------------------|-----------------------------------------------|
| Abstraction        | $A\,\mathbf{x}= \mathbf{b}$                  | $\mathbf{f}(\mathbf{x}) = \mathbf{0}$         |
| Examples in math   | $x_1 + x_2 = 5^2$, $\frac{d^2y}{dx^2} = k y$ | $x_1^2 + x_2^2 = 5^2$, $\frac{dy}{dx} = k y^2 |
| Examples in nature | Mass–spring system at small deflection       | Duffing oscillator, orbital dynamics          |

Linearity is an important concept in modelling, linear models are highly desirable because they are easier to analyse and solve. Linear methods are highly scalable to millions of variables and a great effort is made to find linear model respresentations or to linearize inherently non-linear natural processes. In contrast, non-linear models involve non-linear relationships between variables, making them more complex and often harder to solve. 

Lorem Ipsum


In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

def linear_spring(t, y, k=1.0, m=1.0):
    x, v = y
    return [v, -k/m * x]

def duffing(t, y, alpha=1.0, beta=5.0, delta=0.02, gamma=8.0, omega=1.2):
    x, v = y
    return [v, -delta*v - alpha*x - beta*x**3 + gamma*np.cos(omega*t)]

sol_lin = solve_ivp(linear_spring, [0, 20], [1, 0], max_step=0.05)
sol_non = solve_ivp(duffing,       [0, 20], [1, 0], max_step=0.05)

plt.plot(sol_lin.t, sol_lin.y[0], label="Linear spring")
plt.plot(sol_non.t, sol_non.y[0], label="Duffing")
plt.xlabel("t"); plt.ylabel("x(t)")
plt.legend(); plt.title("Linear vs non-linear response");

ModuleNotFoundError: No module named 'matplotlib'

Linearisation Around an equilibrium $x_0$, expand $f(x)$ in a Taylor series and keep the first derivative:
$f(x)\approx f(x_0)+f'(x_0)(x-x_0)$. You will use this trick repeatedly for control design (Week 11).

### 2.2 Coupled vs. uncoupled
Uncoupled: variables evolve independently.
Example: three orthogonal resistors in an electrical breadboard.

Coupled: state variables interact.
Example: 3-D attitude dynamics where roll, pitch, yaw share inertial products.

(Observe how eigenvalues split when coupling is introduced.)


### 2.3 Symmetry

Lorem Ipsum 
(to be expanded in Week 3 when we discuss conservation laws and Noether’s theorem)


### 2.4 Static vs. dynamic
Static: time is absent; solve for equilibrium.
– Truss analysis in civil engineering
– Steady-state heat conduction

Dynamic: variables evolve in time.
– Launch trajectory optimisation
– Satellite attitude slew manoeuvre

In [ ]:
# Static beam deflection under point load (Euler-Bernoulli):
import sympy as sp
x = sp.symbols('x')
E, I, L, P = sp.symbols('E I L P')
w = (P*x**2)/(6*E*I)*(3*L - x)   # closed-form
sp.simplify(w)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Cantilever beam parameters
L = 2.0          # length in meters
P = 1_000.0      # point load at the free end, in Newtons
E = 200e9        # Young's modulus for steel, in Pascals
I = 1e-6         # second moment of area, in m^4 (slender beam)

# Deflection formula for a cantilever with end load: w(x) = P x^2 (3L - x) / (6 E I)
x = np.linspace(0, L, 250)
w = P * x**2 * (3*L - x) / (6 * E * I)   # meters
w_mm = w * 1e3                            # convert to millimetres for readability

plt.figure()
plt.plot(x, w_mm)
plt.gca().invert_yaxis()  # downward deflection plotted downward
plt.xlabel("Position x (m)")
plt.ylabel("Deflection w (mm)")
plt.title("Cantilever beam deflection under end load (P = 1kN, L = 2m)")
plt.grid(True)
plt.show()
